# Evanston General Election 2017

Let's look at what happened in Evanston's very close election held in April 2017. The Mayor's race was decided by less than 300 votes - a margin of less than 1%! We'll use Folium again to make some chloropleth maps that let us visualize what the results look like in different precincts.

In [1]:
import folium # For maps
import numpy as np # For analysis
import requests # For getting web data
import pandas as pd # For organizing & analysis

In [2]:
# Get the map data from the City of Evanston's open data portal in geoJSON format
geodata = requests.get('http://data-evanston.opendata.arcgis.com/datasets/bde82b5d201b4a36b23e6ca81756e3ba_1.geojson').json()

# Get the election results from the Cook County Elections website
dfs = pd.read_html('http://results417.cookcountyclerk.com/Detail.aspx?eid=40417&rid=103&vfor=1&twpftr=1')

# The raw vote totals are in the fourth dataframe
results = dfs[3]

# Name the columns we want to look at and remove the title and totals rows
results.columns = ['precinct', 'rvs', 'ballots', 'hagerty', 'tendam', 'total']
results = results.iloc[1:-1, :]

The geoJSON document stores the names of the precincts as "W_P" where W is the ward number and P is the precinct number. We've stored that data in the dataframe under the 'w_p' column so we can pass the dataframe to folium and it can match on those names to correctly fill the choropleth.

In [3]:
# Clean up the data by mapping the ward and precinct from the string to the format contained in the 
# geoJSON document so we they can be matched by folium
results = results.apply(lambda x: pd.to_numeric(x, errors='ignore'))
results['ward'] = results['precinct'].apply(lambda x: x.split(' ')[2])
results['sub_precinct'] = results['precinct'].apply(lambda x: x.split(' ')[4])
results['w_p'] = results['ward'] + '-' + results['sub_precinct']

# Calculate voter turnout by total votes as percentage of registered voters in each precinct
results['turnout'] = 100 * results['ballots'] / results['rvs']

# Calculate the share of the vote each candidate received in each precinct
candidates = ['hagerty', 'tendam']
for candidate in candidates:
    results[candidate + '_pct'] = 100 * results[candidate] / results['total']
results['raw_diff'] = results['hagerty'] - results['tendam']
results['pct_diff'] = results['hagerty_pct'] - results['tendam_pct']

In [4]:
# Create a new map centered at Evanston
amap = folium.Map(location=[42.0451, -87.6877],
                  zoom_start=13,
                  tiles='OpenStreetMap')

color_var = 'turnout'

# Create the bin thresholds for differet choropleth colors
thresholds = [0,5,10,20,30,50] #np.linspace(0, results[color_var].max(), 6, dtype=int).tolist()
amap.choropleth(geo_str=geodata,
                data=results,
                columns=['w_p', color_var],
                key_on='feature.properties.W_P',
                fill_color='YlOrRd',
                threshold_scale=thresholds, 
                legend_name='Turnout (%)')
amap

Once again, Northwestern University distinguishes itself as a very low-turnout area. There were, however, areas such as the sixth ward that got higher than 50% turnout! Not a bad showing for an off-year local election.

Now let's look at any geographic variation in the percentage of votes each candidate received in each precinct.

In [5]:
def election_map(quantity_name, thresholds=None, colorscale = 'YlGn'):
    amap = folium.Map(location=[42.0451, -87.6877],
                  zoom_start=13,
                  tiles='OpenStreetMap')

    # Create the bin thresholds for differet choropleth colors
    if(thresholds is None):
        thresholds =  np.linspace(min(0, results[quantity_name].min()), results[quantity_name].max(), 6, dtype=int).tolist()
        
    amap.choropleth(geo_str=geodata,
                    data=results,
                    columns=['w_p', quantity_name],
                    key_on='feature.properties.W_P',
                    fill_color=colorscale,
                    threshold_scale=thresholds, 
                    legend_name=quantity_name)
    
    return amap

It was a close election, so let's change our vote percentage thresholds for each candidate.

In [6]:
pct_thresholds = [40, 45, 50, 55, 60]
election_map('hagerty_pct', thresholds=pct_thresholds, colorscale='YlGn')

In [7]:
election_map('tendam_pct', thresholds=pct_thresholds, colorscale='BuPu')

In [10]:
election_map('pct_diff', thresholds=[-20, -10, 0, 10, 20], colorscale='PRGn')

Interesting stuff! Looks like Steve Hagerty (green) did significantly better in the northern part of Evanston, while Mark Tendam(purple) did better at Northwestern University and in the southern part of Evanston. I don't know how this data correlates with any other demographic variables that might indicate the best predictors of support for either candidate -- for that, we'll have to wait, as always, for more data.